In [4]:
import pandas as pd
import numpy as np
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [12]:
data = pd.read_csv("C:\\Users\\jagadesvar\\Downloads\\data\\complaints.csv")

C:\Users\jagadesvar\AppData\Local\Temp\ipykernel_13624\2805899644.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\jagadesvar\\Downloads\\data\\complaints.csv")


In [13]:
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2024-12-30,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account information incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33073,NaN,Consent not provided,Web,2024-12-30,Closed with non-monetary relief,Yes,NaN,11326072
1,2024-12-30,Credit reporting or other personal consumer re...,Credit reporting,Credit monitoring or identity theft protection...,Problem canceling credit monitoring or identif...,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,77057,NaN,Consent not provided,Web,2024-12-30,Closed with non-monetary relief,Yes,NaN,11326185
2,2024-12-30,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Personal information incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,77057,NaN,Consent not provided,Web,2024-12-30,Closed with non-monetary relief,Yes,NaN,11326105
3,2024-12-30,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Was not notified of investigation status or re...,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,11413,NaN,Consent not provided,Web,2024-12-30,Closed with non-monetary relief,Yes,NaN,11325376
4,2024-12-30,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",WA,98030,NaN,Consent not provided,Web,2024-12-30,Closed with non-monetary relief,Yes,NaN,11325044


In [14]:
data.tail()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
8258537,2016-09-08,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting my free annual report,NaN,NaN,"EQUIFAX, INC.",FL,32853,NaN,Consent not provided,Web,2016-09-08,Closed with explanation,Yes,Yes,2102374
8258538,2022-06-06,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,"EQUIFAX, INC.",CA,93638,NaN,Consent not provided,Web,2022-06-06,Closed with explanation,Yes,NaN,5637858
8258539,2016-07-14,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting report or credit score,NaN,NaN,"EQUIFAX, INC.",NJ,08620,"Older American, Servicemember",NaN,Phone,2016-07-14,Closed with explanation,Yes,No,2012835
8258540,2014-03-03,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,NaN,NaN,"EQUIFAX, INC.",NJ,08759,NaN,NaN,Web,2014-03-04,Closed with explanation,Yes,No,740118
8258541,2014-11-10,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,"EQUIFAX, INC.",CA,95616,NaN,NaN,Postal mail,2014-11-25,Closed with explanation,Yes,No,1110751


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8258542 entries, 0 to 8258541
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    object
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              object
 16  Consumer disputed?            object
 17  Complaint ID                  int64 
dtypes: int64(1), object(17)
memory usage: 1.1+

In [16]:
data.describe(include="all")

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
count,8258542,8258542,8023247,8258536,7435927,2661158,4139825,8258542,8206326,8228314,593866,6817858,8258542,8258542,8258522,8258542,768306,8.258542e+06
unique,4861,21,86,178,272,1957637,11,7559,63,34626,3,4,7,4810,8,2,2,NaN
top,2025-01-17,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,In accordance with the Fair Credit Reporting a...,Company has responded to the consumer and the ...,"EQUIFAX, INC.",FL,XXXXX,Servicemember,Consent not provided,Web,2025-02-25,Closed with explanation,Yes,No,NaN
freq,32842,3623498,5749022,2902696,1854604,19398,3850061,1930069,1066749,156854,365619,3829823,7675367,30838,4856706,8192318,619928,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.233357e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.359577e+06
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.323290e+06
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.602212e+06
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.010266e+07


In [17]:
data.shape

(8258542, 18)

In [18]:
data = data[['Issue', 'Product']]

category mapping

In [19]:
category_mapping = {
    "Credit reporting or other personal consumer reports": 0,
    "Debt collection": 1,
    "Payday loan, title loan, personal loan, or advance loan": 2,
    "Mortgage": 3
}
data['Category'] = data['Product'].map(category_mapping)
data.dropna(inplace=True)

Text Preprocessing

In [20]:
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub(r'\d+', '', text)
    return text

data['Cleaned_Issue'] = data['Issue'].apply(clean_text)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(data['Cleaned_Issue'], data['Category'], test_size=0.2, random_state=42)

Convert text to numeric features

In [24]:
vectorizer = TfidfVectorizer()
X_train_tfidata = vectorizer.fit_transform(X_train)
X_test_tfidata = vectorizer.transform(X_test)

LOGISTIC REGRESSION

In [25]:
LR = LogisticRegression()
LR.fit(X_train_tfidata, y_train)
y_pred = LR.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)



0.9983467308745396


In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    724649
         1.0       1.00      1.00      1.00    146107
         2.0       1.00      0.86      0.93      2704
         3.0       1.00      0.99      0.99     83432

    accuracy                           1.00    956892
   macro avg       1.00      0.96      0.98    956892
weighted avg       1.00      1.00      1.00    956892



In [27]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = LR.predict(sample_vectorized)[0]
    print(f"Logistic Regression Prediction: {prediction}")

# Example sample input
sample_input_LR = "Trouble during payment process"
predict_category(sample_input_LR)

Logistic Regression Prediction: 3.0


In [28]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_LR.csv", index=False)
print("LR_predictions saved")


LR_predictions saved


NAIVE BAYES

In [29]:
NB = MultinomialNB()
NB.fit(X_train_tfidata, y_train)
y_pred = NB.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)



0.9978304761665894


In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    724649
         1.0       1.00      1.00      1.00    146107
         2.0       1.00      0.76      0.87      2704
         3.0       1.00      0.98      0.99     83432

    accuracy                           1.00    956892
   macro avg       1.00      0.94      0.96    956892
weighted avg       1.00      1.00      1.00    956892



In [31]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = NB.predict(sample_vectorized)[0]
    print(f"Naive Bayes Prediction: {prediction}")

# Example sample input
sample_input_NB = "Trouble during payment process"
predict_category(sample_input_NB)

Naive Bayes Prediction: 3.0


In [32]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_NB.csv", index=False)
print("NB_predictions saved")

NB_predictions saved


RANDOM FOREST

In [34]:
RF = RandomForestClassifier(n_estimators=100, random_state=42)
RF.fit(X_train_tfidata, y_train)
y_pred = RF.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


0.998395848225296


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    724649
         1.0       1.00      1.00      1.00    146107
         2.0       1.00      0.88      0.93      2704
         3.0       1.00      0.99      0.99     83432

    accuracy                           1.00    956892
   macro avg       1.00      0.97      0.98    956892
weighted avg       1.00      1.00      1.00    956892



In [36]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = RF.predict(sample_vectorized)[0]
    print(f"Random Forest Prediction: {prediction}")

# Example sample input
sample_input_RF = "Trouble during payment process"
predict_category(sample_input_RF)

Random Forest Prediction: 3.0


In [37]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_RF.csv", index=False)
print("RF_predictions saved")

RF_predictions saved


SUPPORT VECTOR MACHINE

In [38]:
SVM = SVC(kernel='linear')
SVM.fit(X_train_tfidata, y_train)
y_pred = SVM.predict(X_test_tfidata)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9983937581252639


In [39]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    724649
         1.0       1.00      1.00      1.00    146107
         2.0       1.00      0.88      0.93      2704
         3.0       1.00      0.99      0.99     83432

    accuracy                           1.00    956892
   macro avg       1.00      0.97      0.98    956892
weighted avg       1.00      1.00      1.00    956892



In [40]:
def predict_category(sample_text):
    sample_text_cleaned = clean_text(sample_text)
    sample_vectorized = vectorizer.transform([sample_text_cleaned])
    prediction = SVM.predict(sample_vectorized)[0]
    print(f"Support Vector Machines Prediction: {prediction}")

# Example sample input
sample_input_SVM = "Trouble during payment process"
predict_category(sample_input_SVM)


Support Vector Machines Prediction: 3.0


In [41]:
output_data = X_test.to_frame()
output_data['Actual'] = y_test.values
output_data['Predicted'] = y_pred
output_data.to_csv(f"predictions_SVM.csv", index=False)
print("SVM_predictions saved")

SVM_predictions saved
